In [ ]:

https://en.wikipedia.org/wiki/Stiffness_matrix#The_stiffness_matrix_for_the_Poisson_problem


In [1]:
import numpy as np
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import spsolve

In [68]:
def poisson_square_mesh(n):
    # Create a square mesh with n elements per side
    x = np.linspace(0, 2*np.pi, n + 1)
    y = np.linspace(0, 2*np.pi, n + 1)
    X, Y = np.meshgrid(x, y)
    nodes = np.vstack((X.ravel(), Y.ravel())).T

    # Generate element connectivity
    elements = []
    for i in range(n):
        for j in range(n):
            node0 = i * (n + 1) + j
            node1 = (i + 1) * (n + 1) + j
            node2 = (i + 1) * (n + 1) + (j + 1)
            node3 = i * (n + 1) + (j + 1)
            elements.append([node0, node1, node2])
            elements.append([node0, node2, node3])

    # Assemble stiffness matrix and load vector
    num_nodes = (n + 1) * (n + 1)
    A = lil_matrix((num_nodes, num_nodes))
    b = np.zeros(num_nodes)
    
    count=0
    for elem in elements:
        if count==((n*n)/2):
            b[elem]=50
            
        x = nodes[elem, 0]
        y = nodes[elem, 1]

        count=count+1

        # Compute element stiffness matrix
        dx = np.diff(x)+1
        dy = np.diff(y)+1
        area = np.prod(dx) * np.prod(dy)
        B = np.array([
            [1 / dx[0], 1 / dx[1], -1 / dx[1], -1 / dx[0]],
            [1 / dy[0], 1 / dy[1], -1 / dy[1], -1 / dy[0]]
        ])
        K = np.dot(np.dot(B.T, np.diag([1, 1])), B) * area

        # Assemble stiffness matrix and load vector
        for i in range(3):
            for j in range(3):
                A[elem[i], elem[j]] += K[i, j]
        b[elem] += area / 6  # Assuming constant load

    # Apply Dirichlet boundary conditions
    boundary_nodes = np.unique(np.concatenate((
        np.arange(n + 1),                        # Bottom boundary
        np.arange(1, (n + 1) * (n + 1), n + 1),  # Left boundary
        np.arange(n + 1, (n + 1) * (n + 1), n + 1),  # Right boundary
        np.arange(n * (n + 1), (n + 1) * (n + 1))  # Top boundary
    )))

    A[boundary_nodes, :]=1
    for i in range(num_nodes):
        A[boundary_nodes, i] = np.sin(x[i])
    A[boundary_nodes, boundary_nodes] = 1
    b[boundary_nodes] = 0

    # Solve the linear system
    u = spsolve(A.tocsr(), b)

    return nodes, elements, u

In [43]:
def gen_square_mesh(n):
    # Create a square mesh with n elements per side
    x = np.linspace(0, 1, n + 1)
    y = np.linspace(0, 1, n + 1)
    X, Y = np.meshgrid(x, y)
    nodes = np.vstack((X.ravel(), Y.ravel())).T

    # Generate element connectivity
    elements = []
    for i in range(n):
        for j in range(n):
            node0 = i * (n + 1) + j
            node1 = (i + 1) * (n + 1) + j
            node2 = (i + 1) * (n + 1) + (j + 1)
            node3 = i * (n + 1) + (j + 1)
            elements.append([node0, node1, node2])
            elements.append([node0, node2, node3])

    return nodes, elements

In [9]:
def assemble_stiffness_matrix(nodes, elements):
    # Assemble stiffness matrix and load vector
    num_nodes = (n + 1) * (n + 1)
    A = lil_matrix((num_nodes, num_nodes))
    b = np.zeros(num_nodes)

    for elem in elements:

        if elem==
        b[elem]=
        x = nodes[elem, 0]
        y = nodes[elem, 1]

        # Compute element stiffness matrix
        dx = np.diff(x)
        dy = np.diff(y)
        area = np.prod(dx) * np.prod(dy)
        B = np.array([
            [1 / dx[0], 1 / dx[1], -1 / dx[1], -1 / dx[0]],
            [1 / dy[0], 1 / dy[1], -1 / dy[1], -1 / dy[0]]
        ])
        K = np.dot(np.dot(B.T, np.diag([1, 1])), B) * area

        # Assemble stiffness matrix and load vector
        for i in range(3):
            for j in range(3):
                A[elem[i], elem[j]] += K[i, j]
        b[elem] += area / 6  # Assuming constant load
        
    return A, b

In [ ]:
def apply_boundary_conds(A,b,n):
    # Apply Dirichlet boundary conditions
    boundary_nodes = np.unique(np.concatenate((
        np.arange(n + 1),                        # Bottom boundary
        np.arange(1, (n + 1) * (n + 1), n + 1),  # Left boundary
        np.arange(n + 1, (n + 1) * (n + 1), n + 1),  # Right boundary
        np.arange(n * (n + 1), (n + 1) * (n + 1))  # Top boundary
    )))
    A[boundary_nodes, :] = 0
    A[boundary_nodes, boundary_nodes] = 1
    b[boundary_nodes] = 0

    return A,b

In [ ]:
def solve_system(A,b,n):
    # Solve the linear system
    u = spsolve(A.tocsr(), b)
    return u

In [47]:
x=nodes[elements[7],0]
print(x)
print(np.diff(x))

[0.5 1.  1. ]
[0.5 0. ]


In [44]:
n=2
nodes, elements = gen_square_mesh(n)

In [37]:
print(elements)

[[0, 3, 4], [0, 4, 1], [1, 4, 5], [1, 5, 2], [3, 6, 7], [3, 7, 4], [4, 7, 8], [4, 8, 5]]


In [48]:
print(nodes)

[[0.  0. ]
 [0.5 0. ]
 [1.  0. ]
 [0.  0.5]
 [0.5 0.5]
 [1.  0.5]
 [0.  1. ]
 [0.5 1. ]
 [1.  1. ]]


In [10]:
A,b=assemble_stiffness_matrix(nodes, elements)

C:\Users\mikeg\AppData\Local\Temp\ipykernel_30264\3309720924.py:16: RuntimeWarning: divide by zero encountered in scalar divide
  [1 / dx[0], 1 / dx[1], -1 / dx[1], -1 / dx[0]],
C:\Users\mikeg\AppData\Local\Temp\ipykernel_30264\3309720924.py:17: RuntimeWarning: divide by zero encountered in scalar divide
  [1 / dy[0], 1 / dy[1], -1 / dy[1], -1 / dy[0]]
C:\Users\mikeg\AppData\Local\Temp\ipykernel_30264\3309720924.py:19: RuntimeWarning: invalid value encountered in multiply
  K = np.dot(np.dot(B.T, np.diag([1, 1])), B) * area


In [70]:
print(A)

  (0, 0)	nan
  (0, 1)	nan
  (0, 11)	nan
  (0, 12)	nan
  (1, 0)	nan
  (1, 1)	nan
  (1, 2)	nan
  (1, 12)	nan
  (1, 13)	nan
  (2, 1)	nan
  (2, 2)	nan
  (2, 3)	nan
  (2, 13)	nan
  (2, 14)	nan
  (3, 2)	nan
  (3, 3)	nan
  (3, 4)	nan
  (3, 14)	nan
  (3, 15)	nan
  (4, 3)	nan
  (4, 4)	nan
  (4, 5)	nan
  (4, 15)	nan
  (4, 16)	nan
  (5, 4)	nan
  (5, 5)	nan
  (5, 6)	nan
  (5, 16)	nan
  (5, 17)	nan
  (6, 5)	nan
  (6, 6)	nan
  (6, 7)	nan
  (6, 17)	nan
  (6, 18)	nan
  (7, 6)	nan
  (7, 7)	nan
  (7, 8)	nan
  (7, 18)	nan
  (7, 19)	nan
  (8, 7)	nan
  (8, 8)	nan
  (8, 9)	nan
  (8, 19)	nan
  (8, 20)	nan
  (9, 8)	nan
  (9, 9)	nan
  (9, 10)	nan
  (9, 20)	nan
  (9, 21)	nan
  (10, 9)	nan
  (10, 10)	nan
  (10, 21)	nan
  (11, 0)	nan
  (11, 11)	nan
  (11, 12)	nan
  (11, 22)	nan
  (11, 23)	nan
  (12, 0)	nan
  (12, 1)	nan
  (12, 11)	nan
  (12, 12)	nan
  (12, 13)	nan
  (12, 23)	nan
  (12, 24)	nan
  (13, 1)	nan
  (13, 2)	nan
  (13, 12)	nan
  (13, 13)	nan
  (13, 14)	nan
  (13, 24)	nan
  (13, 25)	nan
  (14, 2)	nan
  (1

In [69]:
    n = 50  # Number of elements per side
    nodes, elements, u = poisson_square_mesh(n)
    
    # Plot the solution
    import matplotlib.pyplot as plt
    plt.tripcolor(nodes[:, 0], nodes[:, 1], elements, u, shading='gouraud')
    plt.colorbar()
    plt.title("Finite Element Solution of Poisson Equation")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()

IndexError: index 3 is out of bounds for axis 0 with size 3